In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob
import scipy.io
import os
import sys
import pandas as pd
from PIL import Image

import scipy.io

from defects import *

In [ ]:
pt = r"D:\work\nematics\nematics\nematics\Ken\IO\raw_images\hbec\*tif"

img_lst = glob.glob(pt)
print(img_lst)

In [3]:
# image_list = glob.glob(r"D:\work\nematics\nematics\nematics\_bacteria\hbec_images\raw\test_flow\*tif")
image_list = glob.glob(r"D:\work\nematics\nematics\nematics\Ken\IO\raw_images\hbec\*tif")

from natsort import natsorted
image_list = natsorted(image_list, key=lambda y: y.lower())



In [ ]:
print(type(image_list[0]))

In [ ]:
PLOT  = True
SAVE = False

x, y = 0, 0
win1, win2 = 600, 600
window_size = 40

for img_idx,im in enumerate(image_list):
    
    img = plt.imread(im)
    # img = img[-np.min(img.shape):,-np.min(img.shape):]


    ori, coh, E = orientation_analysis(img, 15)
    k = compute_topological_charges(ori, int_area='cell', origin='upper')
    defects = localize_defects(k, x_grid=xx, y_grid=yy)
    compute_defect_orientations(ori, defects)
    plushalf = defects[defects['charge']==.5]
    minushalf = defects[defects['charge']==-.5]
    
    #### PLOT the image 
    if PLOT:
        
        pix_x = img.shape[1]
        pix_y = img.shape[0]

        x = np.arange(0,pix_x)
        y = np.arange(0,pix_y)
        xx,yy = np.meshgrid(x,y)
        fig, ax  = plt.subplots(figsize=(16,16))
        
        
        s = 15 
        ax.imshow(img, cmap='gray')
        # ax.quiver(xx[::s,::s], yy[::s,::s])
        ax.quiver(xx[::s,::s], yy[::s,::s], 
            np.cos(ori)[::s,::s], np.sin(ori)[::s,::s], 
            headaxislength=0, headwidth=0, headlength=0, 
            color='lawngreen', scale=60, pivot='mid', alpha=.5)

        ax.plot(plushalf['x'], plushalf['y'],'ro',markersize=10,label=r'+1/2 defect')
        ax.quiver(plushalf['x'], plushalf['y'], 
            np.cos(plushalf['ang1']), np.sin(plushalf['ang1']), 
            headaxislength=0, headwidth=0, headlength=0, color='r', scale=50)

        for i in range(3):
            ax.quiver(minushalf['x'], minushalf['y'], 
                np.cos(minushalf['ang'+str(i+1)]), np.sin(minushalf['ang'+str(i+1)]), 
                headaxislength=0, headwidth=0, headlength=0, color='b', scale=50)


    if img_idx>2:
        break




In [ ]:
PLOT  = True
SAVE = False

x, y = 0, 0
win1, win2 = 600, 600

for (i,im1), im2 in zip(enumerate(image_list[:-1]),image_list[1:]):

    img1 = cv2.imread(im1)[:,:,0]
    img2 = cv2.imread(im2)[:,:,0]

    # fig, axs = plt.subplots(2,1)
    # axs = axs.flatten()
    # axs[0].imshow(img1, cmap="gray")
    # axs[1].imshow(img2, cmap="gray")


    flow = cv2.calcOpticalFlowFarneback(img1,img2, None, 0.5, 3, 
        winsize=61, iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

    if PLOT:
        fig = plt.figure(figsize=(15,10))
        step = 15
        plt.imshow(img1, cmap="gray")
        # x, y = np.arange(0, flow.shape[1], step), np.arange(flow.shape[0]-step, -1, -step)
        x = np.arange(0, flow.shape[1], step, dtype=np.int32)
        y = np.arange(0, flow.shape[0], step, dtype=np.int32)
        plt.quiver(x,y, 
                flow[::step, ::step, 0], -flow[::step, ::step, 1], color="red")
        # plt.tight_layout()
        plt.show()
        save_path = os.path.join(os.path.dirname(image_list[i]),'flow_%d.png' % i)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        fig.savefig(save_path)
        plt.cla()
    

    
    if SAVE:
        save_path = os.path.join(
            os.path.dirname(image_list[i]), 
            'OptFlow', 
            os.path.splitext(os.path.basename(image_list[i]))[0] + '.mat'
            )
        # break
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        scipy.io.savemat(save_path, dict(u=flow[:,:,0], v=flow[:,:,1]))
        # "C:\Users\USER\Downloads\B-sub-sur-minus-in-supernatant-40X-100fps\OptFlow\test.mat"


    # break



#test class workflow on 5 images

In [2]:
path =r"D:\work\nematics\nematics\nematics\Ken\IO\raw_images\hbec"
from workflow import *

WF = image_series(path,save = True)
PH,MH = WF.detect_minus_plus()

c:\Users\cheny\anaconda3\envs\tf1\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\cheny\anaconda3\envs\tf1\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


In [10]:
PH

,charge,x,y,x_ind,y_ind,ang1,ang2,ang3,from_img
60,0.5,650.0,51.0,650,51,3.835051,NaN,NaN,0
61,0.5,1846.0,100.0,1846,100,-0.052612,NaN,NaN,0
62,0.5,487.0,135.0,487,135,0.783019,NaN,NaN,0
63,0.5,312.0,147.0,312,147,1.420821,NaN,NaN,0
64,0.5,1398.0,179.0,1398,179,2.324804,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
115,0.5,1724.0,1815.0,1724,1815,3.595425,NaN,NaN,4
116,0.5,1845.0,1862.0,1845,1862,-1.052038,NaN,NaN,4
117,0.5,442.0,1904.0,442,1904,-0.180313,NaN,NaN,4
118,0.5,1815.0,1964.0,1815,1964,4.692753,NaN,NaN,4


In [6]:
# image_list = natsorted(image_list, key=lambda y: y.lower())
# image_list = image_list[::10]
for (i,im1), im2 in zip(enumerate(image_list[:-1]),image_list[1:]):

    methods = ['cv2.TM_CCOEFF']
    pad = 200

    img1 = cv2.imread(im1)[:,:,0]
    img2 = cv2.imread(im2)[:,:,0]
    
    flow = cv2.calcOpticalFlowFarneback(img1,img2, None, 0.5, 3, 
        winsize=61, iterations=3, poly_n=5, poly_sigma=1.2, flags=0)
    fl = flow
    break

In [18]:
WF.path + WF.path.split("\\")[-1]

'D:\\work\\nematics\\nematics\\nematics\\Ken\\IO\\raw_images\\testtest'